Otniel Abiezer - 1301180469

# Run All

## Import dan Load

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn import metrics

In [ ]:
rcParams['figure.figsize'] = 15, 8
mpl.style.use(['ggplot'])

In [ ]:
category = pd.read_excel('https://drive.google.com/uc?id=1dkt0ZuLhiQVdZhMob0oWzHUs_w0DbIzV')
items = pd.read_excel('https://drive.google.com/uc?id=16cBVzweKptXNmtguQALlPhbFSvkSw5sL')
submission = pd.read_excel('https://drive.google.com/uc?id=1BTbDThuS6G4VjJog7raFYK0KpglP2Avx')
order = pd.read_excel('https://drive.google.com/uc?id=19F5OTepUV4H9PL9kEqPGq0BdC5CqanWT')

In [ ]:
category.shape

(4332, 2)

In [ ]:
category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   category         4332 non-null   int64
 1   parent_category  4332 non-null   int64
dtypes: int64(2)
memory usage: 67.8 KB


In [ ]:
items.shape

(32776, 8)

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32776 entries, 0 to 32775
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   itemID      32776 non-null  int64  
 1   brand       32776 non-null  int64  
 2   feature_1   32776 non-null  int64  
 3   feature_2   32776 non-null  int64  
 4   feature_3   32776 non-null  int64  
 5   feature_4   32776 non-null  int64  
 6   feature_5   32776 non-null  int64  
 7   categories  25988 non-null  float64
dtypes: float64(1), int64(7)
memory usage: 2.0 MB


In [ ]:
submission.shape

(10000, 3)

In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   userID      10000 non-null  int64  
 1   itemID      10000 non-null  int64  
 2   prediction  0 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 234.5 KB


In [ ]:
order.shape

(1048575, 4)

In [ ]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   date    1048575 non-null  datetime64[ns]
 1   userID  1048575 non-null  int64         
 2   itemID  1048575 non-null  int64         
 3   order   1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 32.0 MB


## Imputasi Missing Value

In [ ]:
items["categories"].isnull().sum() / items.shape[0] * 100

20.710275811569442

In [ ]:
items["categories"].fillna(-1)

0        2890.0
1          -1.0
2        3270.0
3        3270.0
4        2470.0
          ...  
32771    3502.0
32772    3900.0
32773    2686.0
32774    3900.0
32775    3900.0
Name: categories, Length: 32776, dtype: float64

In [ ]:
items["categories"] = items["categories"].fillna(-1)
items.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,2890.0
1,28640,1366,10,1,537,0,101,-1.0
2,13526,1090,10,0,511,0,0,3270.0
3,21399,1090,10,1,511,0,0,3270.0
4,8504,768,4,1,484,0,66,2470.0


In [ ]:
category.shape[0]

4332

In [ ]:
category.loc[category.shape[0]] = [-1, -1]
category.tail()

,category,parent_category
4328,4296,3898
4329,4297,3898
4330,4298,3898
4331,4299,3898
4332,-1,-1


## Cek duplikasi ItemID dan Category

In [ ]:
items['itemID'].nunique()

32776

In [ ]:
items.shape[0]

32776

In [ ]:
category['category'].nunique()

4301

In [ ]:
category.shape[0]

4333

In [ ]:
category = category.drop_duplicates(subset=['category'], keep='first')
category.shape[0]

4301

## Join 3 Dataframe

In [ ]:
hasil_join = order.join(items.set_index('itemID'), on='itemID').join(category.set_index('category'), on='categories')
hasil_join.head()

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_category
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,74.0,3056
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,3459.0,3056
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,2973.0,566
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,30.0,1682
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,3104.0,1852


In [ ]:
hasil_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1048575 non-null  datetime64[ns]
 1   userID           1048575 non-null  int64         
 2   itemID           1048575 non-null  int64         
 3   order            1048575 non-null  int64         
 4   brand            1048575 non-null  int64         
 5   feature_1        1048575 non-null  int64         
 6   feature_2        1048575 non-null  int64         
 7   feature_3        1048575 non-null  int64         
 8   feature_4        1048575 non-null  int64         
 9   feature_5        1048575 non-null  int64         
 10  categories       1048575 non-null  float64       
 11  parent_category  1048575 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(10)
memory usage: 96.0 MB


In [ ]:
hasil_join.nunique()

date                 242
userID             46127
itemID             31580
order                 51
brand               1481
feature_1             12
feature_2              4
feature_3            540
feature_4              6
feature_5            191
categories          1396
parent_category      411
dtype: int64

## Label (Target)

In [ ]:
target = hasil_join['date'].dt.day
target

0           1
1           1
2           1
3           1
4           1
           ..
1048570    28
1048571    28
1048572    28
1048573    28
1048574    28
Name: date, Length: 1048575, dtype: int64

In [ ]:
def day_to_week(day):
  if (1 <= day <= 7):
    return 1
  elif (8 <= day <= 14):
    return 2
  elif (15 <= day <= 21):
    return 3
  else:
    return 4

In [ ]:
target_week = []

for day in target:
  target_week.append(day_to_week(day))

In [ ]:
hasil_join['Label'] = target_week
hasil_join.tail()

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_category,Label
1048570,2021-01-28,18379,31073,1,1126,4,0,291,3,129,777.0,3027,4
1048571,2021-01-28,18379,11425,1,1445,3,0,-1,-1,-1,1395.0,3233,4
1048572,2021-01-28,80,6024,1,361,10,0,503,0,90,2995.0,3189,4
1048573,2021-01-28,80,29330,1,406,10,0,485,3,132,3444.0,3898,4
1048574,2021-01-28,33107,19304,2,400,10,0,504,3,17,806.0,3027,4


In [ ]:
hasil_join = hasil_join.drop(columns=['date'])
hasil_join.tail()

,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,parent_category,Label
1048570,18379,31073,1,1126,4,0,291,3,129,777.0,3027,4
1048571,18379,11425,1,1445,3,0,-1,-1,-1,1395.0,3233,4
1048572,80,6024,1,361,10,0,503,0,90,2995.0,3189,4
1048573,80,29330,1,406,10,0,485,3,132,3444.0,3898,4
1048574,33107,19304,2,400,10,0,504,3,17,806.0,3027,4


## Drop Kolom yang Tidak Perlu

In [ ]:
hasil_join = hasil_join.drop(columns=['order', 'brand', 'categories', 'userID', 'itemID'])
hasil_join.tail()

,feature_1,feature_2,feature_3,feature_4,feature_5,parent_category,Label
1048570,4,0,291,3,129,3027,4
1048571,3,0,-1,-1,-1,3233,4
1048572,10,0,503,0,90,3189,4
1048573,10,0,485,3,132,3898,4
1048574,10,0,504,3,17,3027,4


## Menggeser Nilai menjadi +1

In [ ]:
hasil_join.min()

feature_1         -1
feature_2          0
feature_3         -1
feature_4         -1
feature_5         -1
parent_category   -1
Label              1
dtype: int64

In [ ]:
hasil_join[['feature_1', 'feature_3', 'feature_4', 'feature_5', 'parent_category']] = hasil_join[['feature_1', 'feature_3', 'feature_4', 'feature_5', 'parent_category']] + 1
hasil_join[['feature_1', 'feature_3', 'feature_4', 'feature_5', 'parent_category']].head()

,feature_1,feature_3,feature_4,feature_5,parent_category
0,7,197,1,46,3057
1,11,230,4,133,3057
2,5,456,1,109,567
3,5,292,4,45,1683
4,7,304,1,46,1853


## Split Train dan Test

In [ ]:
X = hasil_join.iloc[:,:-1]
X.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,parent_category
0,7,0,197,1,46,3057
1,11,3,230,4,133,3057
2,5,1,456,1,109,567
3,5,0,292,4,45,1683
4,7,0,304,1,46,1853


In [ ]:
y = hasil_join.iloc[:,-1:]
y.head()

,Label
0,1
1,1
2,1
3,1
4,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, stratify=y)

## Pembangunan Model

In [ ]:
cnb = CategoricalNB()
cnb.fit(X_train, y_train.values.ravel())

CategoricalNB()

## Evaluasi

In [ ]:
prediksi = cnb.predict(X_test)

In [ ]:
print("Accuracy : ", metrics.accuracy_score(y_test, prediksi))

Accuracy :  0.29571533203125


In [ ]:
metrics.confusion_matrix(y_test, prediksi)

array([[ 1483,  1439,   882, 55454],
       [ 1548,  1624,   965, 59110],
       [ 1574,  1495,  1049, 57486],
       [ 1688,  1814,  1169, 73364]])